# 과정

1. 데이터 가져오기
2. 데이터 전처리

3. 모델 구성
    - Layer
        - Convolution
        > Output Ch., Activation Func., Pooling, Dropout

        - Fully Connected (DNN)
        > Flatten(Reshape), Dense(Output Ch.), Activation Func., Dropout
        
    - Param
        - Loss
            - (Sparse) Categorical CrossEntropy / Binary Cross Entropy
            > 오차 계산 방식

        - Optimizer
            - SGD / RMS Prop / Adam
            > 기여율 계산 방식
        
        - Metrics
            > Accuracy 계산 방식 / (맞은 개수 / 전체 개수)
            
   - Compile
    > 모델 구성에 Param 선언까지 해주기

4. 모델 학습
    - Hyper Parameter
        - learning rate 
            > 계산된 오차, 기여율을 얼만큼 Scaling해서 Weight를 조정해줄지
        - batch size 
            > 한번에 얼만큼의 이미지를 넣을것인지
        - epoch 
            > 학습과정을 얼마나 반복할것인지

5. 평가
    - Test Data
        - Back Propagation, Gradient 계산 불필요 (bcz. 단순 예측, 라벨 비교)
        - argmax(dim, keepdim)
            > Softmax 되어 나온 확률 중 가장 큰 값의 라벨 도출
        

# Tensorflow
--------------
## 데이터 불러오기

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import tensorflow as tf
from  tensorflow.keras import datasets, layers

mnist = datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data("mnist.npz")

In [2]:
input_shape = (28, 28, 1)
num_label = 10
dropout_rate = 0.5

## 모델 구성

In [3]:
# Convolution

inputs = layers.Input(shape=input_shape)

net = layers.Conv2D(32, 3, padding='SAME')(inputs)
# 출력 채널(width, num_filter) = 32
# 커널 크기(filter) = (3, 3)
net = layers.Activation('relu')(net)

net = layers.Conv2D(32, 3, padding = "SAME")(net)
net = layers.Activation('relu')(net)

net = layers.MaxPool2D((2, 2))(net) # (batch_size, 14, 14, 32)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, 3, padding = "SAME")(net)
net = layers.Activation('relu')(net)

net = layers.Conv2D(64, 3, padding = "SAME")(net)
net = layers.Activation('relu')(net)

net = layers.MaxPool2D((2, 2))(net) # (batch_size, 7, 7, 64)
net = layers.Dropout(0.25)(net)



# Fully Connected (DNN)

net = layers.Flatten()(net)  # (1, 3136)
net = layers.Dense(512)(net) # (1, 512)
net = layers.Activation('relu')(net)

net = layers.Dropout(dropout_rate)(net)

net = layers.Dense(num_label)(net)
net = layers.Activation('softmax')(net)


# 모델 이름 설정
model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

## Parameter 및 compile

In [4]:
learning_rate = 0.001
num_epochs = 1
batch_size = 64

In [5]:
optimizer = tf.keras.optimizers.Adam(learning_rate)
loss = 'sparse_categorical_crossentropy'
metrics = ['accuracy']

model.compile(optimizer, loss, metrics)

## 데이터 전처리

In [6]:
x_train, x_test = x_train[..., tf.newaxis], x_test[..., tf.newaxis]

x_train, x_test = x_train / 255., x_test / 255.

## 모델 학습

In [7]:
history = model.fit(x_train, y_train, batch_size, num_epochs, shuffle = True)

938/938 [==============================] - 187s 200ms/step - loss: 0.1858 - accuracy: 0.9413


## 모델 평가

In [8]:
model.evaluate(x_test, y_test, batch_size)

157/157 [==============================] - 5s 34ms/step - loss: 0.0411 - accuracy: 0.9865


[0.04110949486494064, 0.9865000247955322]

# Pytorch
-----------

In [9]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 

# 데이터 관리를 위한 라이브러리
from torchvision import datasets, transforms

In [10]:
seed = 1

lr = 0.001
momentum = 0.5

batch_size = 64
test_batch_size = 64

epochs = 2

device = torch.device("cuda")

log_interval = 100

## 데이터 가져오기

In [11]:
torch.manual_seed(seed)

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train = True, download = True,
                    transform = transforms.Compose([
                        transforms.ToTensor(), # 데이터를 Tensor 형식으로
                        transforms.Normalize((0.1307, ), (0.3081, )) # 데이터 표준화 
                    ])),
    batch_size = batch_size, shuffle = True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset', train=False,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307, ), (0.3081, ))])),
    batch_size = test_batch_size, shuffle = True)

## 모델 구성

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1) #input ch., output ch., kernel size, stri
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        
        self.fc1 = nn.Linear(4*4*50, 500) # (Torch) Linear = (TF) Dense
        self.fc2 = nn.Linear(500, 10)
    
    def forward(self, x):
        x = F.relu(self.conv1(x)) # 커널 생성 후, Activation 함수 적용
        x = F.max_pool2d(x, 2, 2) # pool_size, stride
        
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)

        x = x.reshape(-1, 4 * 4 * 50)
        
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return F.log_softmax(x, dim=1)

model = Net().to(device) # device에 모델 컴파일

## (Torch)Optimizer 및 Compile
(TF) Optimizer, Loss, metrics

In [13]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
# model.parameters = 모델을 구성하고있는 층 별 shape를 확인할수있음 (Weight, Bias 등)

## 모델 학습

In [14]:
for epoch in range(1, epochs + 1):
    model.train() # train모드로 변경

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target) # 오차 계산
        loss.backward() # backward로 영향력 계산
        optimizer.step() # 얼만큼 뺄지 계산

        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\t Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100 * batch_idx / len(train_loader), loss.item()
            ))
    

    model.eval() # evaluation 모드로 변경
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100 * correct / len(test_loader.dataset)
    ))


Train Epoch: 1 [0/60000 (0%)]	 Loss: 2.292215
Train Epoch: 1 [6400/60000 (11%)]	 Loss: 2.231636
Train Epoch: 1 [12800/60000 (21%)]	 Loss: 2.151003
Train Epoch: 1 [19200/60000 (32%)]	 Loss: 1.913002
Train Epoch: 1 [25600/60000 (43%)]	 Loss: 1.640912
Train Epoch: 1 [32000/60000 (53%)]	 Loss: 1.259970
Train Epoch: 1 [38400/60000 (64%)]	 Loss: 0.862546
Train Epoch: 1 [44800/60000 (75%)]	 Loss: 0.898350
Train Epoch: 1 [51200/60000 (85%)]	 Loss: 0.610933
Train Epoch: 1 [57600/60000 (96%)]	 Loss: 0.739944

Test set: Average loss: 0.0077, Accuracy: 8589/10000 (86%)n
Train Epoch: 2 [0/60000 (0%)]	 Loss: 0.443154
Train Epoch: 2 [6400/60000 (11%)]	 Loss: 0.533468
Train Epoch: 2 [12800/60000 (21%)]	 Loss: 0.402233
Train Epoch: 2 [19200/60000 (32%)]	 Loss: 0.407886
Train Epoch: 2 [25600/60000 (43%)]	 Loss: 0.338484
Train Epoch: 2 [32000/60000 (53%)]	 Loss: 0.377512
Train Epoch: 2 [38400/60000 (64%)]	 Loss: 0.327293
Train Epoch: 2 [44800/60000 (75%)]	 Loss: 0.432609
Train Epoch: 2 [51200/60000 (85%)

# Matplotlib 관련은 09.ipynb 확인